In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime

In [ ]:
class RealTimeDataIngestion:
    def __init__(self, api_key, city):
        self.api_key = api_key
        self.city = city
        self.base_url = "http://api.openweathermap.org/data/2.5/weather"
    
    def fetch_data(self):
        params = {
            'q': self.city,
            'appid': self.api_key,
            'units': 'metric',  # Temperature in Celsius
        }
        response = requests.get(self.base_url, params=params)
        data = response.json()
        if data.get("cod") != 200:
            print("Error fetching data from API:", data.get("message"))
            return None
        return data
    
class DataTransformation:
    def transform(self, raw_data):
        if raw_data is None:
            return None
        
        # Extract relevant fields
        weather_info = {
            'city': raw_data['name'],
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'temperature': raw_data['main']['temp'],
            'humidity': raw_data['main']['humidity'],
            'pressure': raw_data['main']['pressure'],
            'weather_description': raw_data['weather'][0]['description'],
        }
        
        # Convert to DataFrame
        df = pd.DataFrame([weather_info])
        return df

class RealTimeMetricsCalculation:
    def calculate(self, data_frame):
        # Example: Calculate moving averages or other metrics (in real-time)
        data_frame['temperature_avg'] = data_frame['temperature'].rolling(window=2).mean()
        return data_frame

class DataStorage:
    def store(self, data_frame, file_path):
        data_frame.to_csv(file_path, mode='a', header=False, index=False)


In [ ]:
class RealTimePipeline:
    def __init__(self, api_key, city, file_path):
        self.api_key = api_key
        self.city = city
        self.file_path = file_path
        
    def run(self):
        # 1. Real-Time Data Ingestion
        ingestion = RealTimeDataIngestion(self.api_key, self.city)
        raw_data = ingestion.fetch_data()
        
        # 2. Data Transformation
        transformation = DataTransformation()
        transformed_data = transformation.transform(raw_data)
        
        # 3. Real-Time Metrics Calculation or Data Processing
        metrics = RealTimeMetricsCalculation()
        metrics_data = metrics.calculate(transformed_data)
        
        # 4. Data Storage
        storage = DataStorage()
        storage.store(metrics_data, self.file_path)
        
        print("Data stored successfully at:", self.file_path)

if __name__ == "__main__":
    # API Key for OpenWeather
    api_key = "OPenWeatherAPIKey"  # Replace with your API key
    # city = 'London'  # You can change this to the city of your choice
    city = 'Amsterdam' 
    
    # File path where we store the data (could be a cloud storage path)
    file_path = 'data/real_time_weatherData_Pandas.csv'

    # Initialize and run the pipeline every minute (for example)
    pipeline = RealTimePipeline(api_key, city, file_path)
    while True:
        pipeline.run()
        time.sleep(10)  # Fetch new data every 10 seconds